## Classification

In this notebook we will see some basic classification methods:
- Logistic regression
- Support vector machine
- Decision tree
- Random forests


We will use the following libraries:
- `sklearn`  main library for machine learning algorithms
- `scipy`
- `pandas`
- `numpy`
- `matplotlib`
- `seaborn`




In [20]:
from sklearn import datasets
from sklearn import linear_model, svm, tree
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from scipy.cluster.hierarchy import dendrogram, linkage 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seabornas sns

import random 
random.seed(42)
%matplotlib inline




## The breast cancer data
 

In [35]:
# Load dataset
data = datasets.load_breast_cancer()
print(data.DESCR)

Breast Cancer Wisconsin (Diagnostic) Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, field
        13 is Radius SE, field 23 is Worst Radius.

        

With this dataset, we will try to classify the  tumor as begnin or malignant, the target variable

In [36]:
cancer_x=pd.DataFrame(data['data'],columns=data.feature_names) #these will be x in our model
cancer_y=data.target # these will be y in our model

In [48]:
cancer=pd.DataFrame(data['data'],columns=data.feature_names)
cancer["cancer10"]=data.target   # save cancer as another column in the dataset with 0 and 1
cancer["cancer10"]=data.target_names[data.target]   # save cancer as another column in the dataset with begnin or malignant
cancer

mean radius  mean texture  mean perimeter  mean area  mean smoothness  \
0         17.990         10.38          122.80     1001.0          0.11840   
1         20.570         17.77          132.90     1326.0          0.08474   
2         19.690         21.25          130.00     1203.0          0.10960   
3         11.420         20.38           77.58      386.1          0.14250   
4         20.290         14.34          135.10     1297.0          0.10030   
5         12.450         15.70           82.57      477.1          0.12780   
6         18.250         19.98          119.60     1040.0          0.09463   
7         13.710         20.83           90.20      577.9          0.11890   
8         13.000         21.82           87.50      519.8          0.12730   
9         12.460         24.04           83.97      475.9          0.11860   
10        16.020         23.24          102.70      797.8          0.08206   
11        15.780         17.89          103.60      781.0          0.09710   
12        19.170         24.80          132.40     1123.0          0.09740   
13        15.850         23.95          103.70      782.7          0.08401   
14        13.730         22.61           93.60      578.3          0.11310   
15        14.540         27.54           96.73      658.8          0.11390   
16        14.680         20.13           94.74      684.5          0.09867   
17        16.130         20.68          108.10      798.8          0.11700   
18        19.810         22.15          130.00     1260.0          0.09831   
19        13.540         14.36           87.46      566.3          0.09779   
20        13.080         15.71           85.63      520.0          0.10750   
21         9.504         12.44           60.34      273.9          0.10240   
22        15.340         14.26          102.50      704.4          0.10730   
23        21.160         23.04          137.20     1404.0          0.09428   
24        16.650         21.38          110.00      904.6          0.11210   
25        17.140         16.40          116.00      912.7          0.11860   
26        14.580         21.53           97.41      644.8          0.10540   
27        18.610         20.25          122.10     1094.0          0.09440   
28        15.300         25.27          102.40      732.4          0.10820   
29        17.570         15.05          115.00      955.1          0.09847   
..           ...           ...             ...        ...              ...   
539        7.691         25.44           48.34      170.4          0.08668   
540       11.540         14.44           74.65      402.9          0.09984   
541       14.470         24.99           95.81      656.4          0.08837   
542       14.740         25.42           94.70      668.6          0.08275   
543       13.210         28.06           84.88      538.4          0.08671   
544       13.870         20.70           89.77      584.8          0.09578   
545       13.620         23.23           87.19      573.2          0.09246   
546       10.320         16.35           65.31      324.9          0.09434   
547       10.260         16.58           65.85      320.8          0.08877   
548        9.683         19.34           61.05      285.7          0.08491   
549       10.820         24.21           68.89      361.6          0.08192   
550       10.860         21.48           68.51      360.5          0.07431   
551       11.130         22.44           71.49      378.4          0.09566   
552       12.770         29.43           81.35      507.9          0.08276   
553        9.333         21.94           59.01      264.0          0.09240   
554       12.880         28.92           82.50      514.3          0.08123   
555       10.290         27.61           65.67      321.4          0.09030   
556       10.160         19.59           64.73      311.7          0.10030   
557        9.423         27.88           59.26      271.3          0.08123   
558       14.590         22

In [5]:
cancer.describe()

mean radius  mean texture  mean perimeter    mean area  \
count   569.000000    569.000000      569.000000   569.000000   
mean     14.127292     19.289649       91.969033   654.889104   
std       3.524049      4.301036       24.298981   351.914129   
min       6.981000      9.710000       43.790000   143.500000   
25%      11.700000     16.170000       75.170000   420.300000   
50%      13.370000     18.840000       86.240000   551.100000   
75%      15.780000     21.800000      104.100000   782.700000   
max      28.110000     39.280000      188.500000  2501.000000   

       mean smoothness  mean compactness  mean concavity  mean concave points  \
count       569.000000        569.000000      569.000000           569.000000   
mean          0.096360          0.104341        0.088799             0.048919   
std           0.014064          0.052813        0.079720             0.038803   
min           0.052630          0.019380        0.000000             0.000000   
25%           0.086370          0.064920        0.029560             0.020310   
50%           0.095870          0.092630        0.061540             0.033500   
75%           0.105300          0.130400        0.130700             0.074000   
max           0.163400          0.345400        0.426800             0.201200   

       mean symmetry  mean fractal dimension           ...             \
count     569.000000              569.000000           ...              
mean        0.181162                0.062798           ...              
std         0.027414                0.007060           ...              
min         0.106000                0.049960           ...              
25%         0.161900                0.057700           ...              
50%         0.179200                0.061540           ...              
75%         0.195700                0.066120           ...              
max         0.304000                0.097440           ...              

       worst radius  worst texture  worst perimeter   worst area  \
count    569.000000     569.000000       569.000000   569.000000   
mean      16.269190      25.677223       107.261213   880.583128   
std        4.833242       6.146258        33.602542   569.356993   
min        7.930000      12.020000        50.410000   185.200000   
25%       13.010000      21.080000        84.110000   515.300000   
50%       14.970000      25.410000        97.660000   686.500000   
75%       18.790000      29.720000       125.400000  1084.000000   
max       36.040000      49.540000       251.200000  4254.000000   

       worst smoothness  worst compactness  worst concavity  \
count        569.000000         569.000000       569.000000   
mean           0.132369           0.254265         0.272188   
std            0.022832           0.157336         0.208624   
min            0.071170           0.027290         0.000000   
25%            0.116600           0.147200         0.114500   
50%            0.131300           0.211900         0.226700   
75%            0.146000           0.339100         0.382900   
max            0.222600           1.058000         1.252000   

       worst concave points  worst symmetry  worst fractal dimension  
count            569.000000      569.000000               569.000000  
mean               0.114606        0.290076                 0.083946  
std                0.065732        0.061867                 0.018061  
min                0.000000        0.156500                 0.055040  
25%                0.064930        0.250400                 0.071460  
50%                0.099930        0.282200                 0.080040  
75%                0.161400        0.317900                 0.092080  
max                0.291000        0.663800                 0.207500  

[8 rows x 30 columns]

In [6]:
sns.heatmap(cancer.corr())

Based on the result above, and some data exploration, describe some patterns in the breast dataset

## Classification

### Creating dataset

The first part is to create a training and test dataset, this can be done automatically with the `sklearn` library. we will set 20% of the data apart for training

In [7]:
train_x, test_x,train_y,test_y = train_test_split(cancer_x,cancer_y,test_size=0.20, random_state=40)

In [61]:
train_x.count()[0]

1437

In [9]:
test_x.count()[0]

114

### Logistic regression



Logistic Regression is a type of Generalized Linear Model (GLM) that model a binary response variable $y$. Here $X$ is all the explanatory variables and $y$ is the tumor type.

1) Build the model with the `train` dataset

In [10]:
LR = linear_model.LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(train_x, train_y)  

2) Predict the classification with the test dataset 

In [11]:
LR.predict(test_x)  

array([1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1])

3) Measure the accuracy of the prediction with this model on the test dataset

In [37]:
round(LR.score(train_x,train_y), 4) 

0.9473

In [38]:
LR.score(test_x,test_y)


0.9385964912280702

## Support Vector Machine

Support-vector machines are supervised learning models with associated learning algorithms that analyze data used for classification and regression analysis. 

They work very similarly to logistic regression:

1) Fit the model to the `train` dataset

In [40]:
SVM = svm.LinearSVC().fit(train_x, train_y)  

2) Predict the classification with the test dataset 

In [41]:
SVM.predict(test_x)  

array([1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1])

3) Measure the accuracy of the prediction with this model on the test dataset

In [42]:
SVM.score(test_x,test_y)

0.9473684210526315

## Decision trees

Decision trees are model based on a tree structure with successive split from an inferred set of variables. The `DecisionTreeClassifier` implements a CART algorithm.


In [13]:
clf= tree.DecisionTreeClassifier().fit(train_x,train_y)

In [14]:
clf.predict(test_x)

array([1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1])

In [15]:
clf.score(test_x,test_y)

0.9473684210526315

To plot the tree try: (there may be conflict in the libraries)

In [17]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

dot_data = StringIO()

export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

InvocationException: GraphViz's executables not found

## Random Forests

Random forest are methods for classification, regression and other tasks that operates by constructing a multitude of decision trees 

1) Build model

In [43]:
RF = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0).fit(train_x, train_y)  

2) Predict

In [44]:
RF.predict(test_x) 

array([1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 1])

3) Measure accuracy

In [45]:
RF.score(test_x,test_y)

0.9473684210526315

## Comparison

For each model write the accuracy to compare them:
- logistic regression: 0.9385964912280702
- SVM: 0.9473684210526315
- Decision tree: 0.9473684210526315
- Random forest: 0.9473684210526315

## Exercise

Using the digits dataset from `sklearn` perform a classification to infer the digits

In [69]:
print(data_digits.DESCR)

Optical Recognition of Handwritten Digits Data Set

Notes
-----
Data Set Characteristics:
    :Number of Instances: 5620
    :Number of Attributes: 64
    :Attribute Information: 8x8 image of integer pixels in the range 0..16.
    :Missing Attribute Values: None
    :Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)
    :Date: July; 1998

This is a copy of the test set of the UCI ML hand-written digits datasets
http://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where
each class refers to a digit.

Preprocessing programs made available by NIST were used to extract
normalized bitmaps of handwritten digits from a preprinted form. From a
total of 43 people, 30 contributed to the training set and different 13
to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of
4x4 and the number of on pixels are counted in each block. This generates
an input matrix of 8x8 where each element is a

In [70]:
data_digits=datasets.load_digits()
digits_x=pd.DataFrame(data_digits['data']) #these will be x in our model
digits_y=data_digits.target # these will be y in our model
train_x, test_x,train_y,test_y = train_test_split(digits_x,digits_y,test_size=0.20, random_state=40)


In [63]:
LR = linear_model.LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(train_x, train_y)  
LR.score(test_x,test_y)

0.9611111111111111

In [64]:
SVM = svm.LinearSVC().fit(train_x, train_y) 
SVM.score(test_x,test_y)

0.95

In [65]:
clf= tree.DecisionTreeClassifier().fit(train_x,train_y)
clf.score(test_x,test_y)

0.8305555555555556

In [66]:
RF = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0).fit(train_x, train_y)  
RF.score(test_x,test_y)

0.7527777777777778

For each model write the accuracy to compare them:
- logistic regression: 0.9611111111111111
- SVM: 0.95
- Decision tree: 0.8305555555555556
- Random forest: 0.7527777777777778